In [ ]:
"""
Test Your Trained Deepfake Detection Model - BALANCED VERSION
Fast processing + Shows only TOP 10 largest suspicious regions!
"""

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import io
import os
import json
from scipy import ndimage
from scipy.ndimage import binary_opening, find_objects
import joblib

# ===== CONFIGURATION =====
# Update these paths to match your model files
MODEL_DIR = 'feature_model_output'
MODEL_PATH = os.path.join(MODEL_DIR, 'random_forest_model.pkl')
SCALER_PATH = os.path.join(MODEL_DIR, 'feature_scaler.pkl')
CONFIG_PATH = os.path.join(MODEL_DIR, 'feature_config.json')

# Image to test (CHANGE THIS TO YOUR IMAGE PATH)
TEST_IMAGE_PATH = r'/Users/ahnjh51/Python/deepfakev2/test008.jpg'

# Display setting
TOP_N_BOXES = 10  # Show only the top N largest suspicious regions

# ===== LOAD MODEL =====
print("=" * 70)
print("LOADING TRAINED MODEL")
print("=" * 70)

try:
    rf_model = joblib.load(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    with open(CONFIG_PATH, 'r') as f:
        config = json.load(f)
    
    ELA_QUALITY = config['ela_quality']
    THRESHOLD_MULTIPLIER = config['threshold_multiplier']
    MIN_REGION_SIZE = config['min_region_size']
    ENHANCEMENT_STRENGTH = config['enhancement_strength']
    feature_names = config['feature_names']
    
    print("✓ Model loaded successfully!")
    print(f"✓ Configuration loaded: Quality={ELA_QUALITY}, Threshold={THRESHOLD_MULTIPLIER}")
    print(f"✓ Display setting: Showing top {TOP_N_BOXES} largest suspicious regions")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nMake sure you have these files in the 'feature_model_output' folder:")
    print("  - random_forest_model.pkl")
    print("  - feature_scaler.pkl")
    print("  - feature_config.json")
    exit(1)

# ===== ELA FUNCTIONS =====
def perform_ela(image_path, quality=95, enhancement=2.5):
    """Perform ELA on image"""
    img = Image.open(image_path).convert('RGB')
    
    buffer = io.BytesIO()
    img.save(buffer, 'JPEG', quality=quality)
    buffer.seek(0)
    compressed = Image.open(buffer)
    
    ela = ImageChops.difference(img, compressed)
    extrema = ela.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    
    scale = 255.0 / max_diff
    ela = ImageEnhance.Brightness(ela).enhance(scale)
    ela = ImageEnhance.Brightness(ela).enhance(enhancement)
    ela = ImageEnhance.Contrast(ela).enhance(enhancement)
    
    return np.array(ela.convert('L')), img

def find_suspicious_regions(ela_array, threshold, min_area=50):
    """
    Find suspicious regions in ELA image - BALANCED VERSION
    Fast processing + keeps legitimate red boxes!
    """
    suspicious_mask = ela_array > threshold
    
    # Count suspicious pixels for progress
    num_suspicious = np.sum(suspicious_mask)
    print(f"  → Suspicious pixels: {num_suspicious:,} ({num_suspicious/ela_array.size*100:.2f}%)")
    
    if num_suspicious == 0:
        return [], [], []
    
    # SMART OPTIMIZATION: Only apply noise removal if there are TOO MANY suspicious pixels
    # This indicates noisy real images, not edited images
    noise_threshold_pct = 15  # If more than 15% pixels are suspicious, it's likely noise
    
    if (num_suspicious / ela_array.size * 100) > noise_threshold_pct:
        print(f"  → High noise detected, applying light filtering...")
        # Use LIGHTER morphological opening (smaller structure, 1 iteration)
        structure = np.ones((2, 2))  # Smaller 2x2 instead of 3x3
        suspicious_mask = binary_opening(suspicious_mask, structure=structure, iterations=1)
        print(f"  → After noise removal: {np.sum(suspicious_mask):,} pixels")
    else:
        print(f"  → Low noise, skipping filtering (keeping all suspicious regions)")
    
    # Label connected regions
    labeled_array, num_features = ndimage.label(suspicious_mask)
    print(f"  → Found {num_features:,} regions, filtering by size...")
    
    if num_features == 0:
        return [], [], []
    
    # Speed optimization: If too many regions (> 1000), use find_objects
    # Otherwise, use the original method for better accuracy
    if num_features > 1000:
        print(f"  → Many regions detected, using fast method...")
        slices = find_objects(labeled_array)
        
        boxes = []
        region_sizes = []
        region_intensities = []
        
        for i, slice_obj in enumerate(slices, 1):
            if slice_obj is None:
                continue
            
            region_mask = labeled_array[slice_obj] == i
            region_size = np.sum(region_mask)
            
            if region_size < min_area:
                continue
            
            y_slice, x_slice = slice_obj
            y_min, y_max = y_slice.start, y_slice.stop
            x_min, x_max = x_slice.start, x_slice.stop
            width = x_max - x_min
            height = y_max - y_min
            
            if width > 10 and height > 10:
                boxes.append((x_min, y_min, width, height))
                region_sizes.append(region_size)
                region_values = ela_array[slice_obj][region_mask]
                region_intensities.append(np.mean(region_values))
    else:
        # Use original method for better bounding boxes
        print(f"  → Using precise method for bounding boxes...")
        boxes = []
        region_sizes = []
        region_intensities = []
        
        for i in range(1, num_features + 1):
            coords = np.argwhere(labeled_array == i)
            
            if len(coords) < min_area:
                continue
            
            y_min, x_min = coords.min(axis=0)
            y_max, x_max = coords.max(axis=0)
            width = x_max - x_min
            height = y_max - y_min
            
            if width > 10 and height > 10:
                boxes.append((x_min, y_min, width, height))
                region_sizes.append(len(coords))
                region_values = ela_array[labeled_array == i]
                region_intensities.append(np.mean(region_values))
    
    print(f"  ✓ Kept {len(boxes)} regions after filtering")
    
    return boxes, region_sizes, region_intensities

def extract_features(image_path):
    """Extract comprehensive features from ELA analysis"""
    try:
        print("\n  Performing ELA analysis...")
        ela_array, original_img = perform_ela(image_path, quality=ELA_QUALITY, enhancement=ENHANCEMENT_STRENGTH)
        
        # Basic statistics
        mean_error = np.mean(ela_array)
        max_error = np.max(ela_array)
        std_error = np.std(ela_array)
        median_error = np.median(ela_array)
        
        # Threshold
        threshold = mean_error + (THRESHOLD_MULTIPLIER * std_error)
        print(f"  → Threshold: {threshold:.2f}")
        
        # Suspicious regions
        suspicious_pixels = np.sum(ela_array > threshold)
        suspicious_percentage = (suspicious_pixels / ela_array.size) * 100
        
        # Find regions (balanced function)
        print(f"  Finding suspicious regions...")
        boxes, region_sizes, region_intensities = find_suspicious_regions(
            ela_array, threshold, min_area=MIN_REGION_SIZE
        )
        
        num_regions = len(boxes)
        
        # Region statistics
        if num_regions > 0:
            avg_region_size = np.mean(region_sizes)
            max_region_size = np.max(region_sizes)
            avg_region_intensity = np.mean(region_intensities)
            region_size_std = np.std(region_sizes)
        else:
            avg_region_size = 0
            max_region_size = 0
            avg_region_intensity = 0
            region_size_std = 0
        
        # Histogram features
        hist, _ = np.histogram(ela_array, bins=10, range=(0, 255))
        hist_norm = hist / ela_array.size
        
        # Percentile features
        p25 = np.percentile(ela_array, 25)
        p50 = np.percentile(ela_array, 50)
        p75 = np.percentile(ela_array, 75)
        p90 = np.percentile(ela_array, 90)
        p95 = np.percentile(ela_array, 95)
        
        # Edge detection features
        edges_h = np.abs(np.diff(ela_array, axis=0)).sum()
        edges_v = np.abs(np.diff(ela_array, axis=1)).sum()
        
        # Combine all features
        features = np.concatenate([
            [mean_error, max_error, std_error, median_error, threshold, 
             suspicious_pixels, suspicious_percentage],
            [num_regions, avg_region_size, max_region_size, 
             avg_region_intensity, region_size_std],
            hist_norm,
            [p25, p50, p75, p90, p95],
            [edges_h, edges_v]
        ])
        
        return features, ela_array, threshold, boxes, region_sizes, original_img
    
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None, None, None, None, None, None

# ===== TEST IMAGE =====
print("\n" + "=" * 70)
print("ANALYZING IMAGE")
print("=" * 70)
print(f"Image: {TEST_IMAGE_PATH}")

if not os.path.exists(TEST_IMAGE_PATH):
    print(f"\n❌ ERROR: Image not found at {TEST_IMAGE_PATH}")
    print("\nPlease update TEST_IMAGE_PATH in the script to point to your image.")
    exit(1)

# Extract features
import time
start_time = time.time()

features, ela_array, threshold, boxes, region_sizes, original_img = extract_features(TEST_IMAGE_PATH)

elapsed_time = time.time() - start_time

if features is None:
    print("❌ Failed to extract features from image")
    exit(1)

print(f"\n✓ Features extracted in {elapsed_time:.2f} seconds: {len(features)} features")

# Scale features
features_scaled = scaler.transform(features.reshape(1, -1))

# Make prediction
prediction = rf_model.predict(features_scaled)[0]
prediction_proba = rf_model.predict_proba(features_scaled)[0]

# ===== RESULTS =====
print("\n" + "=" * 70)
print("🎯 PREDICTION RESULTS")
print("=" * 70)

result = "FAKE" if prediction == 1 else "REAL"
confidence = prediction_proba[prediction] * 100

print(f"\n{'🚨' if prediction == 1 else '✅'} Prediction: {result}")
print(f"📊 Confidence: {confidence:.2f}%")
print(f"   - Real: {prediction_proba[0]*100:.2f}%")
print(f"   - Fake: {prediction_proba[1]*100:.2f}%")

print(f"\n📈 ELA Analysis:")
print(f"   - Suspicious Regions: {len(boxes)}")
print(f"   - Suspicious Pixels: {(features[6]):.2f}%")
print(f"   - Mean ELA Error: {features[0]:.2f}")
print(f"   - Max ELA Error: {features[1]:.2f}")
if len(boxes) > 0:
    print(f"   - Largest Region: {max(region_sizes)} pixels")
    print(f"   - Average Region: {np.mean(region_sizes):.0f} pixels")

# ===== SELECT TOP N LARGEST BOXES =====
# Keep ALL boxes for accurate model prediction
# But only DISPLAY the top N largest boxes
boxes_with_sizes = list(zip(boxes, region_sizes))
boxes_sorted = sorted(boxes_with_sizes, key=lambda x: x[1], reverse=True)  # Sort by size
top_boxes = boxes_sorted[:TOP_N_BOXES]  # Take top N

print(f"\n📦 Box Selection:")
print(f"   - Total regions detected: {len(boxes)} (used for model)")
print(f"   - Top {min(TOP_N_BOXES, len(boxes))} largest shown: {len(top_boxes)}")
print(f"   - Smaller regions hidden: {len(boxes) - len(top_boxes)}")

# ===== VISUALIZATION =====
print("\n" + "=" * 70)
print("GENERATING VISUALIZATION")
print("=" * 70)

fig = plt.figure(figsize=(18, 6))

# Original image with red boxes (TOP N ONLY)
ax1 = plt.subplot(1, 3, 1)
img_with_boxes = original_img.copy()
draw = ImageDraw.Draw(img_with_boxes)

# Draw only TOP N boxes
for box, size in top_boxes:
    x, y, w, h = box
    draw.rectangle([x, y, x+w, y+h], outline='red', width=5)

ax1.imshow(img_with_boxes)
ax1.set_title(f'Original Image + Top {len(top_boxes)} Suspicious Regions\n(out of {len(boxes)} total detected)', 
              fontsize=14, fontweight='bold')
ax1.axis('off')

# ELA image
ax2 = plt.subplot(1, 3, 2)
ax2.imshow(ela_array, cmap='jet')
ax2.set_title('ELA Heatmap\n(Brighter = More Suspicious)', 
              fontsize=14, fontweight='bold')
ax2.axis('off')

# ELA with boxes (TOP N ONLY)
ax3 = plt.subplot(1, 3, 3)
ax3.imshow(ela_array, cmap='jet')
for box, size in top_boxes:
    x, y, w, h = box
    rect = plt.Rectangle((x, y), w, h, fill=False, 
                         edgecolor='red', linewidth=3)
    ax3.add_patch(rect)
ax3.set_title(f'ELA + Top {len(top_boxes)} Red Boxes\n(Largest suspicious areas)', 
              fontsize=14, fontweight='bold')
ax3.axis('off')

# Add prediction text
fig.text(0.5, 0.95, f'Prediction: {result} ({confidence:.1f}% confidence)', 
         ha='center', fontsize=20, fontweight='bold',
         color='red' if prediction == 1 else 'green',
         bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', linewidth=2))

plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.savefig('test_result.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualization saved as 'test_result.png'")

# ===== DETAILED BOX STATISTICS =====
if len(top_boxes) > 0:
    print("\n" + "=" * 70)
    print(f"📊 TOP {len(top_boxes)} LARGEST SUSPICIOUS REGIONS")
    print("=" * 70)
    
    print(f"\nRanked by size (largest first):")
    for i, (box, size) in enumerate(top_boxes, 1):
        x, y, w, h = box
        print(f"  {i}. Position: ({x}, {y}), Size: {w}×{h}, Area: {size:,} pixels")

# ===== FEATURE IMPORTANCE FOR THIS IMAGE =====
print("\n" + "=" * 70)
print("TOP 5 FEATURES FOR THIS IMAGE")
print("=" * 70)

feature_importance = rf_model.feature_importances_
indices = np.argsort(feature_importance)[::-1]

print("\nMost important features and their values:")
for i in range(min(5, len(feature_names))):
    idx = indices[i]
    print(f"{i+1}. {feature_names[idx]:<25} = {features[idx]:.2f} (importance: {feature_importance[idx]:.4f})")

# ===== ADJUSTMENT GUIDE =====
print("\n" + "=" * 70)
print("📝 ADJUSTMENT GUIDE")
print("=" * 70)
print(f"\nCurrent setting: TOP_N_BOXES = {TOP_N_BOXES}")
print(f"  Result: Showing {len(top_boxes)} boxes out of {len(boxes)} detected")

print(f"\n💡 To adjust, change TOP_N_BOXES at the top of the code:")
print(f"  • TOP_N_BOXES = 5  → Show only top 5 largest")
print(f"  • TOP_N_BOXES = 10 → Show top 10 (current)")
print(f"  • TOP_N_BOXES = 20 → Show top 20")
print(f"  • TOP_N_BOXES = {len(boxes)} → Show all boxes")

print("\n" + "=" * 70)
print(f"✅ ANALYSIS COMPLETE! (Total time: {elapsed_time:.2f} seconds)")
print("=" * 70)

In [4]:
pip install uvicorn

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
